In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn
import sys
import os
import time
import scipy
import pickle
from importlib import reload
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from sklearn.metrics import r2_score
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error
from collections import namedtuple, defaultdict
import keras_tuner as kt 
codebase_path = '/data/home/wpw035/Codebase'
sys.path.insert(0, codebase_path) #add path to my codebase models
_train_test_path = 'train_test_inds/Phos'

In [ ]:
#home_path = '/data/home/wpw035/Drug_response_prediction/DRP-alpha-preliminary-results'

_train_test_path = 'train_test_inds/Phos'

In [2]:
from DRP_utils import data_preprocessing as dp_nb
reload(dp_nb)
from DRP_utils import model_selection as ms_nb
reload(ms_nb)
from DRP_utils import testing as t_nb
reload(t_nb)
import Data_imports as di_nb
reload(di_nb)
import pairs_train_test_split as tts_nb
import Learning_curve as lc_nb

In [3]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10
num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

In [4]:
#read in landmark genes for fs and find landmarks that overlap with rna data
landmark_genes = pd.read_csv(
    f'{codebase_path}/downloaded_data_small/landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), rna.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create input data for each drug
x_all, x_drug, y_list = dp_nb.create_all_drugs(
    rna[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

x_all = x_all.astype(np.float32)
x_drug = x_drug.astype(np.float16)

#fmt index to include drug cell line paris
cls_drugs_index = x_all.index + '::' + x_drug.index
x_all.index = cls_drugs_index
x_drug.index = cls_drugs_index
y_list.index = cls_drugs_index

x_all.shape, x_drug.shape, len(y_list)

((11583, 908), (11583, 345), 11583)

In [6]:
_train_size  = 0.6 #train size relative to total data set size
lg_space = np.logspace(1, np.log2(64), base=2.0, num=5).astype(int)
lg2 = np.logspace(np.log2(64), np.log2(len(x_all) * _train_size),  base=2.0, num=50).astype(int)
lg_space = np.concatenate((lg_space, lg2))
lg_space = np.unique(lg_space)
lg_space

array([   2,    4,   11,   26,   64,   70,   77,   85,   93,  103,  113,
        125,  137,  151,  166,  183,  201,  221,  244,  268,  295,  325,
        358,  394,  433,  477,  525,  577,  635,  699,  769,  847,  932,
       1025, 1128, 1242, 1366, 1503, 1654, 1821, 2003, 2205, 2426, 2669,
       2937, 3232, 3557, 3914, 4307, 4740, 5215, 5739, 6315, 6949])

In [7]:
#make mean model
class Mean_model():
    def __init__(self, y_train):
        model = defaultdict(list)
        #group cls by drugs
        for ind, val in y_train.iteritems():
            cl, d = ind.split('::')
            model[d].append(val)
            
        #take average of all values for a given drug   
        for d in _all_drugs:
            model[d] = np.mean(model[d])
        
        self.model = model
        
    def predict(self, y_index, reformat=True):
        #reformat index to get just drug
        if reformat:
            y_index = [y.split('::')[1] for y in y_index]
            
        return np.array([self.model[y] for y in y_index])
    
    def replace_nan(self, re='mean'):
        #replace nan's with re, deflat re=0
        for k in self.model:
            if np.isnan(self.model[k]):
                if re=='mean':
                    vals = np.array(list(self.model.values()))
                    vals = vals[~np.isnan(vals)]
                    self.model[k] = vals.mean()
                else:
                    self.model[k] = re
    

In [11]:
#create LC's
for run in range(30):
    test = pd.read_csv(f'{_train_test_path}/test_inds{run}', 
                       dtype=str, delimiter='/n',header=None)[0].values
    train = pd.read_csv(f'{_train_test_path}/train_inds{run}', 
                        dtype=str, delimiter='/n',header=None)[0].values
    y_train = y_list.loc[train]
    y_test = y_list.loc[test]

    test_results = {'r2': [], 'mse': [], 'rho' : []}
    for train_size in lg_space:

        #find y subset and create mean model
        y_train_sub = y_train[:train_size]
        mm = Mean_model(y_train_sub)
        #replace nan's from drugs that are not in the train set with zeros
        mm.replace_nan()
        pre = mm.predict(y_test.index)

        #find performance metrics
        test_results['rho'].append(
            scipy.stats.pearsonr(y_test, pre))
        test_results['r2'].append(
            sklearn.metrics.r2_score(y_test, pre))
        test_results['mse'].append(
            sklearn.metrics.mean_squared_error(y_test, pre))
    test_results = pd.DataFrame(test_results)
    test_results['train size'] = lg_space
    test_results.to_csv(f'LC-metric-results/Mean_model/run{run}')

/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/data/home/wpw035/.conda/envs/tfGPUforge/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data/home/wpw035/.conda/envs/tfGPUforge/lib/